In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive, files
drive.mount("/content/drive")
drive_path = "/content/drive/My Drive/CSCI-576"

In [ ]:
def read_image(filename):
    img = cv2.imread(f"{drive_path}/{filename}", cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"File not found: {drive_path}/{filename}")
    assert img.dtype == np.uint8
    return img.astype(np.float32) / 255.0

In [ ]:
def write_image(M, filename):
    if M.ndim != 2:
        raise ValueError("Matrix must be 2-dimensional")
    img = np.clip(M, 0.0, 1.0) * 255.0
    img = img.astype(np.uint8)
    if not cv2.imwrite(f"{drive_path}/{filename}", img): #, cv2.IMWRITE_JPEG_QUALITY, 100
        raise IOError(f"Error writing to file {drive_path}/{filename}")

In [ ]:
def display_image(M, title):
    ROWS, COLUMNS = M.shape
    if M.ndim != 2:
        raise ValueError("Matrix must be 2-dimensional")
    img = np.clip(M, 0.0, 1.0)
    plt.figure(figsize=(ROWS / 128, COLUMNS / 128), dpi=128)
    plt.imshow(img, cmap='gray', vmin=0.0, vmax=1.0)
    if title: plt.title(f"{title} {ROWS}x{COLUMNS}")
    else: plt.title(f"{ROWS}x{COLUMNS}")
    plt.axis('off')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
image_names = [
    "grayMan"
  , "grayMan_gaussian"
  , "grayMan_saltandpepper"
  , "Q4_original"
  , "Q4_corrupted"
]
for name in image_names:
    M = read_image(f"{name}.png")
    print(M.shape)

In [ ]:
# Question 1
#-------------------------------------------------------------------------------
M = read_image("grayMan.png")
ROWS, COLUMNS = M.shape
R = -1
while R < 0 or R > ROWS:
    try: R = int(input(f"Number of rows up to {ROWS}:")) # <== input 204
    except: pass
C = -1
while C < 0 or C > COLUMNS:
    try: C = int(input(f"Number of columns up to {COLUMNS}: ")) # <== input 306
    except: pass
rows = np.random.choice(np.arange(0, ROWS), size=R, replace=False)
columns = np.random.choice(np.arange(0, COLUMNS), size=C, replace=False)
S = M
S = np.delete(S, rows, axis=0)
S = np.delete(S, columns, axis=1)
display_image(M, f"Original")
display_image(S, f"Sliced Up!")
write_image(S, f"grayMan_1.jpg")

In [ ]:
# Question 2
def correlation(img, transform):
    padded = np.pad(img, pad_width=1, mode='edge')
    output = np.zeros_like(img)
    for r in range(ROWS):
        for c in range(COLUMNS):
            output[r, c] = transform(padded, r, c)
    return output

In [ ]:
# Question 2
laplacian_kernel      = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
mega_laplacian_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
average_kernel        = np.array([[1/9, 1/9, 1/9], [1/9, 1/9, 1/9], [1/9, 1/9, 1/9]])
sobel_x_kernel        = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
sobel_y_kernel        = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
if True:
    from scipy.signal import medfilt2d
    from scipy.signal import correlate2d
    def median_smoothing_filter(I):     return medfilt2d(I, kernel_size=3)
    def average_smoothing_filter(I):    return correlate2d(I, average_kernel,   mode='same', boundary='symm').clip(0.0, 1.0)
    def laplacian_sharpening_filter(I): return correlate2d(I, laplacian_kernel, mode='same', boundary='symm').clip(0.0, 1.0)
    def mega_laplacian_filter(I):       return correlate2d(I, mega_laplacian_kernel,  mode='same', boundary='symm').clip(0.0, 1.0)
    def sobel_x_filter(I):              return correlate2d(I, sobel_x_kernel,   mode='same', boundary='symm').clip(0.0, 1.0)
    def sobel_y_filter(I):              return correlate2d(I, sobel_y_kernel,   mode='same', boundary='symm').clip(0.0, 1.0)
else:
    def median_smoothing_filter(I):     return correlation(I, lambda I, r, c: np.median(I[r:r+3, c:c+3]))
    def average_smoothing_filter(I):    return correlation(I, lambda I, r, c: np.sum(I[r:r+3, c:c+3] * average_kernel)).clip(0.0, 1.0)
    def laplacian_sharpening_filter(I): return correlation(I, lambda I, r, c: np.sum(I[r:r+3, c:c+3] * laplacian_kernel)).clip(0.0, 1.0)
    def mega_laplacian_filter(I):       return correlation(I, lambda I, r, c: np.sum(I[r:r+3, c:c+3] * mega_laplacian_kernel)).clip(0.0, 1.0)
    def sobel_x_filter(I):              return correlation(I, lambda I, r, c: np.sum(I[r:r+3, c:c+3] * sobel_x_kernel)).clip(0.0, 1.0)
    def sobel_y_filter(I):              return correlation(I, lambda I, r, c: np.sum(I[r:r+3, c:c+3] * sobel_y_kernel)).clip(0.0, 1.0)

In [ ]:
# Question 2
#-------------------------------------------------------------------------------
names = {"grayMan_gaussian": 'g', "grayMan_saltandpepper": 'sb'}
for name in names:
    original      = read_image(f"{name}.png")
    avg_smooth    = average_smoothing_filter(original)
    avg_laplacian = laplacian_sharpening_filter(avg_smooth)
    med_smooth    = median_smoothing_filter(original)
    med_laplacian = laplacian_sharpening_filter(med_smooth)
    display_image(original,       f"Original {name}")
    display_image(avg_smooth,     f"Avg smoothing {name}")
    display_image(avg_laplacian,  f"Avg smooth+laplacian {name}")
    display_image(med_smooth,     f"Median smoothing {name}")
    display_image(med_laplacian,  f"Median smooth+laplacian {name}")
    write_image(avg_smooth,       f"fnA{names[name]}.jpg")
    write_image(avg_laplacian,    f"fnA{names[name]}L.jpg")
    write_image(med_smooth,       f"fnM{names[name]}.jpg")
    write_image(med_laplacian,    f"fnM{names[name]}L.jpg")

In [ ]:
# Comparison
#   average smoothing works nicely with gaussian noise, but not at all with impulse noise
#   Laplacian with smoothed gaussian noise brings back detail but also brings back the noise
#   median smoothing also works well with gaussian noise, hard to tell which is better
#   median smoothing works very nicely with impulse noise
#   Laplacian with smoothed impulse noise looks very nice

In [ ]:
# Question 3
def histogram_display(hist):
    bins = np.arange(256)
    plt.figure(figsize=(6, 4))
    plt.bar(bins, hist, width=1.0, color='gray', alpha=0.75)
    plt.title("Histogram")
    plt.xlabel('Intensity')
    plt.ylabel('Occurrences')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
# Question 3
def histogram_calculate(img):
    hist = [0] * 256
    ROWS, COLUMNS = img.shape
    for r in range(ROWS):
        for c in range(COLUMNS):
            assert img[r, c] >= 0.0 and img[r, c] <= 1.0
            hist[int(img[r, c] * 255)] += 1
    return hist

In [ ]:
# Question 3
def CDF_calculate(hist):
    cdf = [0] * 256
    cdf[0] = hist[0]
    for i in range(1, 256):
        cdf[i] = cdf[i - 1] + hist[i]
    return cdf

In [ ]:
# Question 3
def histogram_equalization(img):
    ROWS, COLUMNS = img.shape
    total = ROWS * COLUMNS
    hist = histogram_calculate(img)
    cdf = CDF_calculate(hist)
    cdf_min = cdf[0]
    for i in range(256):
        cdf[i] = (cdf[i] - cdf_min) / (total - cdf_min)
    out = np.zeros_like(img)
    for i in range(ROWS):
        for j in range(COLUMNS):
            out[i, j] = cdf[int(img[i, j] * 255)]
    return out

In [ ]:
# Question 3
def power_transform(x, gamma): return x ** gamma
def log_transform(x, c): return c * np.log(1.0 + x)
def inverse_log_transform(x, c): return c * (np.exp(x) - 1.0)
def minmax_stretch_transform(x, min, max): return (x - min) / (max - min)

In [ ]:
# Question 3(a)
#-------------------------------------------------------------------------------
# This image was chosen due to having many areas with dark shadows
name = "Forest"
POWER = 0.55
original = read_image("Q3_forest.png")
output = power_transform(original, POWER)
display_image(original, f"{name} Original")
display_image(output, f"{name} Power={POWER}")
write_image(output, f"Q3_forest_3a.png")

In [ ]:
# Question 3(b)
#-------------------------------------------------------------------------------
# First image chosen due to its histogram limited to one region
names = ["Mushrooms", "Hawkes Bay"]
filenames = ["Q3_mushrooms", "Q3_unequalized_hawkes_bay_nz"]
extnames = ["png", "jpg"]
for n in range(0, 2): # 0, 2
    original = read_image(f"{filenames[n]}.{extnames[n]}")
    output = histogram_equalization(original)
    display_image(original, f"{names[n]} Original")
    display_image(output, f"{names[n]} Histogram EQ")
    write_image(output, f"{filenames[n]}_3b.png")

In [ ]:
# Question 3(c)
#-------------------------------------------------------------------------------
# First image chosen due to its histogram limited with multiple clusters
names = ["Moon", "Misty Mountains"]
filenames = ["Q3_sphx_glr_plot_scientific_005", "Q3_misty_mountains"]
extnames = ["png", "jpg"]
for n in range(0, 2):
    if n == 0: XSLIDE, ROW_BLOCKS, COLUMN_BLOCKS = 4, 16, 20
    if n == 1: XSLIDE, ROW_BLOCKS, COLUMN_BLOCKS = 4, 16, 24
    original = read_image(f"{filenames[n]}.{extnames[n]}")
    ROWS, COLUMNS = original.shape
    display_image(original, f"{names[n]} Original")
    output = np.zeros_like(original, dtype=np.float32)
    weighting = np.zeros_like(original, dtype=np.float32)
    accumulator = np.zeros_like(original, dtype=np.float32)
    height = ROWS // ROW_BLOCKS
    width = COLUMNS // COLUMN_BLOCKS
    for r in range(ROW_BLOCKS * XSLIDE):
        for c in range(COLUMN_BLOCKS * XSLIDE):
            r_start = ((r * height) // XSLIDE); r_end = r_start + height
            c_start = ((c * width) // XSLIDE);  c_end = c_start + width
            weighting[r_start:r_end, c_start:c_end] += 1.0
            accumulator[r_start:r_end, c_start:c_end] += \
                histogram_equalization(original[r_start:r_end, c_start:c_end])
    output = accumulator / weighting
    display_image(output, f"{names[n]} Localized Histogram EQ")
    write_image(output, f"{filenames[n]}_3c.png")

In [ ]:
# Question 3(d)
#-------------------------------------------------------------------------------
# Image chosen since demonstrates a proven technique,
# i.e. using logarithm on fourier spectrum
name = "Spectrum"
original = read_image("Q3_spectrum.png")
output = log_transform(original, 4.0)
display_image(original, f"{name} Original")
display_image(output, f"{name} Logarithm")
write_image(output, "Q3_spectrum_3d.png")

In [ ]:
# Question 3(e)
#-------------------------------------------------------------------------------
# Image chosen since it had narrow range of intensities to expand
name = "Bread Rolls"
original = read_image("Q3_bread rolls.png")
output = minmax_stretch_transform(original, np.min(original), np.max(original))
display_image(original, f"{name} Original")
display_image(output, f"{name} Linear Contrast Stretch")
write_image(output, "Q3_bread rolls_3e.png")

In [ ]:
# Question 4
from skimage.metrics import structural_similarity as ssim
def compare_images(original, restored):
    if original.shape != restored.shape:
        raise ValueError(f"Mismatched shapes: {original.shape} != {restored.shape}")
    data_range = original.max() - original.min()
    if data_range == 0: data_range = 255.0
    mse = np.mean((original - restored) ** 2)
    if mse == 0: psnr = float('inf')
    else: psnr = 20 * np.log10(data_range) - 10 * np.log10(mse)
    ssim_metric, diff_map = ssim(original, restored, full=True, data_range=data_range)
    print(f"MSE={mse}", f"PSNR={psnr}", f"simularity={ssim_metric}") # , diff_map

In [ ]:
# Question 4
from scipy.ndimage import gaussian_filter
def unsharp_masking(img, radius=1.0, amount=1.0):
    blurred = gaussian_filter(img, sigma=radius)
    return np.clip(img + amount * (img - blurred), 0.0, 1.0)

In [ ]:
def sobel_sharpening(img, amount=1.0):
    sobel_x = sobel_x_filter(img)
    sobel_y = sobel_x_filter(img)
    edges = np.hypot(sobel_x, sobel_y)
    return np.clip(img + amount * edges, 0.0, 1.0)

In [ ]:
# Question 4
#-------------------------------------------------------------------------------
# Possibly necessitates deblurring techniques beyond current scope of class.
# The following sequence achieved 0.57 simularity to the original
name = "Q4"
original = read_image("Q4_original.png")
corrupted = read_image("Q4_corrupted.png")
MIN = np.min(corrupted)
MAX = np.max(corrupted)
steps = [
    ("3x3", lambda img: median_smoothing_filter(img)), # Nice
#   ("1x1", log_transform, 1.0),
#   ("1x1", power_transform, 0.50),
    ("1x1", minmax_stretch_transform, MIN, MAX), # Nice
#   ("NxN", lambda img: histogram_equalization(img)),
#   ("3x3", lambda img: laplacian_sharpening_filter(img)),
#   ("3x3", lambda img: mega_laplacian_filter(img)),
    ("NxN", unsharp_masking, 1.0, 1.0), # tiny help
#   ("NxN", sobel_sharpening, 2.0),
    ("3x3", lambda img: average_smoothing_filter(img)), # little help
]
restored = corrupted
for step in steps:
#   histogram_display(histogram_calculate(restored))
    match step[0]:
        case "1x1": restored = (step[1])(restored, *step[2:]).clip(0.0, 1.0)
        case "NxN": restored = (step[1])(restored, *step[2:])
        case "3x3": restored = (step[1])(restored)
compare_images(original, original)
compare_images(original, corrupted)
compare_images(original, restored)
display_image(restored, f"{name} Restored")
display_image(original, f"{name} Original")
write_image(restored, "Q4_corrupted_4.png")